In [1]:
import datetime as dt
import numpy as np
import pandas as pd
from scipy.linalg import cho_solve

import matplotlib.pyplot as plt
%matplotlib inline

lib_path = r'/Users/rmfeng/jupyter/AC-FE/'
import sys
sys.path.append(lib_path)
import acfe_methods as am
import imp_vol_approx as apx

from importlib import reload

### Part iv) Solutions of Solved ImpVol and Approximations

In [11]:
df.iloc[:,-4:]

,c_ivol,p_ivol,c_ivol_approx,p_ivol_approx
Strike,,,,
2150.0,0.173169,0.173398,0.170789,0.171020
2175.0,0.168303,0.168558,0.166377,0.166636
2200.0,0.163491,0.163586,0.161981,0.162078
2225.0,0.158719,0.158758,0.157582,0.157622
2250.0,0.153714,0.153786,0.152903,0.152975
2275.0,0.148798,0.148901,0.148263,0.148366
2300.0,0.143922,0.143975,0.143610,0.143663
2325.0,0.139030,0.139113,0.138884,0.138967
2350.0,0.134209,0.134174,0.134167,0.134132


In [2]:
S = 2381.0
val_date = dt.date(2017, 3, 16)
mat_date = dt.date(2017, 9, 29)
T = np.busday_count(val_date, mat_date) / (365.25 * 5/7)

In [3]:
fn = r"S&P500_ETF_Option_0917.xlsx"
df = pd.read_excel(fn, header=1)[1:]
df.set_index('Strike', inplace=True)
df.rename(columns={
    'Ticker':'c_ticker',
    'Bid':'c_bid',
    'Ask':'c_ask',
    'Volm':'c_volm',
    'Strike.1':'strike',
    'Ticker.1':'p_ticker',
    'Bid.1':'p_bid',
    'Ask.1':'p_ask',
    'Volm.1':'p_volm'
}, inplace=True)
df['c_px'] = (df.c_bid + df.c_ask) / 2
df['p_px'] = (df.p_bid + df.p_ask) / 2

In [5]:
ones = [1] * len(df.strike)
neg_strike = -df.strike.values
A = np.column_stack([ones, neg_strike])
y = (df.c_px - df.p_px).values.reshape(-1,1)
AtA = np.matmul(A.transpose(), A)
Aty = np.matmul(A.transpose(), y)
Ut = np.linalg.cholesky(AtA)
x = cho_solve((Ut, True), Aty)
PVF = x[0]
disc = x[1]
q = (-np.log(PVF / S) * (1/T))[0]
r = (-np.log(disc) * (1/T))[0]

#### Implied Vols

In [6]:
df.head()

,c_ticker,c_bid,c_ask,c_volm,strike,p_ticker,p_bid,p_ask,p_volm,c_px,p_px
Strike,,,,,,,,,,,
2150.0,SPXW 9/29/17 C2150,259.299988,260.700012,0.0,2150.0,SPXW 9/29/17 P2150,34.799999,35.700001,10.0,260.000000,35.250000
2175.0,SPXW 9/29/17 C2175,238.100006,239.600006,0.0,2175.0,SPXW 9/29/17 P2175,38.500000,39.400002,10.0,238.850006,38.950001
2200.0,SPXW 9/29/17 C2200,217.399994,218.899994,0.0,2200.0,SPXW 9/29/17 P2200,42.500000,43.500000,10.0,218.149994,43.000000
2225.0,SPXW 9/29/17 C2225,197.199997,198.699997,0.0,2225.0,SPXW 9/29/17 P2225,47.099998,48.099998,0.0,197.949997,47.599998
2250.0,SPXW 9/29/17 C2250,177.399994,178.899994,0.0,2250.0,SPXW 9/29/17 P2250,52.099998,53.200001,1.0,178.149994,52.650000


In [7]:
c_ivol_list = np.array([])
p_ivol_list = np.array([])

for row in df.iterrows():
    K = row[1]['strike']
    c_px = row[1]['c_px']
    p_px = row[1]['p_px']
    
    c_ivol = am.imp_vol_newton(c_px, 0.2, T, True, S, K, r, q, am.norm_cdf_def_thresh)
    p_ivol = am.imp_vol_newton(p_px, 0.2, T, False, S, K, r, q, am.norm_cdf_def_thresh)
    
    c_ivol_list = np.append(c_ivol_list, c_ivol)
    p_ivol_list = np.append(p_ivol_list, p_ivol)

df['c_ivol'] = c_ivol_list
df['p_ivol'] = p_ivol_list

#### Using paper vol approximator

In [9]:
c_vol_approx_list = np.array([])
p_vol_approx_list = np.array([])

for row in df.iterrows():
    K = row[1]['strike']
    c_px = row[1]['c_px']
    p_px = row[1]['p_px']
    c_ivol_approx = apx.impvol_approx_call(c_px, K, T, S, r, q)
    p_ivol_approx = apx.impvol_approx_put(p_px, K, T, S, r, q)
    c_vol_approx_list = np.append(c_vol_approx_list, c_ivol_approx)
    p_vol_approx_list = np.append(p_vol_approx_list, p_ivol_approx)

df['c_ivol_approx'] = c_vol_approx_list
df['p_ivol_approx'] = p_vol_approx_list